# Persistent RAG System on Google Colab

This notebook implements a RAG system that **saves its knowledge base to Google Drive**.
This means you only have to process your PDFs/Text files once. Subsequent runs will load the existing database.

### **Features:**
1. **Google Drive Integration**: Mounts your drive to save data.
2. **Smart Loading**: Checks if a vector DB exists before creating a new one.
3. **API Server**: Exposes the chat API via ngrok for your Mobile App.

In [10]:
# --- 1. INSTALL DEPENDENCIES ---
!pip install -q -U requests==2.32.4 "opentelemetry-sdk<1.39.0" transformers accelerate bitsandbytes langchain langchain-community sentence-transformers chromadb pypdf unstructured networkx flask flask-cors pyngrok

Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/cli/base_command.py", line 179, in exc_logging_wrapper
    status = run_func(*args)
             ^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/cli/req_command.py", line 67, in wrapper
    return func(self, options, args)
           ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/commands/install.py", line 377, in run
    requirement_set = resolver.resolve(
                      ^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/resolution/resolvelib/resolver.py", line 95, in resolve
    result = self._result = resolver.resolve(
                            ^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/pip/_vendor/resolvelib/resolvers.py", line 546, in resolve
    state = resolution.resolve(requirements, max_rounds=max_rounds)
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

In [17]:
# --- 2. IMPORTS ---
import os
import torch
from google.colab import drive
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import DirectoryLoader, PyPDFLoader, TextLoader
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_community.llms import HuggingFacePipeline
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from flask import Flask, request, jsonify
from flask_cors import CORS
from pyngrok import ngrok
import threading
import shutil

In [12]:
# Local (SAFE for writes)
LOCAL_MODEL_PATH = "/content/models"
LOCAL_DB_PATH = "/content/chroma_db"

# Drive (BACKUP ONLY)
DRIVE_ROOT = "/content/drive/MyDrive/rag_smart_parking"
DRIVE_DB_PATH = os.path.join(DRIVE_ROOT, "chroma_db")
DATA_PATH = os.path.join(DRIVE_ROOT, "data")

os.makedirs(LOCAL_MODEL_PATH, exist_ok=True)
os.makedirs(LOCAL_DB_PATH, exist_ok=True)
os.makedirs(DATA_PATH, exist_ok=True)


In [13]:
# --- 4. INITIALIZE EMBEDDINGS ---
EMBEDDING_MODEL_NAME = "sentence-transformers/all-MiniLM-L6-v2"
print("Loading Embedding Model...")
embeddings = HuggingFaceEmbeddings(
    model_name=EMBEDDING_MODEL_NAME,
    model_kwargs={'device': 'cuda'}
)

Loading Embedding Model...


In [14]:
# Always work locally
if os.path.exists(LOCAL_DB_PATH) and os.listdir(LOCAL_DB_PATH):
    print("✅ Loading existing LOCAL Chroma DB")
    vector_store = Chroma(
        persist_directory=LOCAL_DB_PATH,
        embedding_function=embeddings
    )

else:
    print("⚠️ Creating new Chroma DB")

    # Demo doc if empty
    if not any(f.endswith((".txt", ".pdf")) for f in os.listdir(DATA_PATH)):
        with open(os.path.join(DATA_PATH, "demo.txt"), "w") as f:
            f.write(
                "Smart Parking System: users book parking spots via mobile app. "
                "Payments supported: ABA PayWay. Admin dashboard monitors slots."
            )

    docs = []
    docs.extend(
        DirectoryLoader(DATA_PATH, glob="**/*.txt", loader_cls=TextLoader).load()
    )
    docs.extend(
        DirectoryLoader(DATA_PATH, glob="**/*.pdf", loader_cls=PyPDFLoader).load()
    )

    splitter = RecursiveCharacterTextSplitter(
        chunk_size=500,
        chunk_overlap=50
    )
    chunks = splitter.split_documents(docs)

    vector_store = Chroma.from_documents(
        documents=chunks,
        embedding=embeddings,
        persist_directory=LOCAL_DB_PATH
    )

    vector_store.persist()
    print("✅ Chroma DB created locally")


⚠️ Creating new Chroma DB
✅ Chroma DB created locally


/tmp/ipython-input-3818644737.py:40: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vector_store.persist()


In [18]:
if os.path.exists(DRIVE_DB_PATH):
    shutil.rmtree(DRIVE_DB_PATH)

shutil.copytree(LOCAL_DB_PATH, DRIVE_DB_PATH)
print("✅ Chroma DB backed up to Drive")


✅ Chroma DB backed up to Drive


In [19]:
# --- 6. LOAD LLM (Mistral/Zephyr) ---
MODEL_NAME = "HuggingFaceH4/zephyr-7b-beta"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

# Path for Model Cache
MODEL_CACHE_PATH = os.path.join(DRIVE_ROOT, "models")
os.makedirs(MODEL_CACHE_PATH, exist_ok=True)

print(f"Loading {MODEL_NAME}...")
print(f"(First run will download to {MODEL_CACHE_PATH}, subsequent runs will be fast)")
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, cache_dir=MODEL_CACHE_PATH)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    quantization_config=bnb_config,
    device_map="auto",
    cache_dir=MODEL_CACHE_PATH
)

text_generation_pipeline = pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    temperature=0.2,
    do_sample=True,
    repetition_penalty=1.1,
    return_full_text=False,
    max_new_tokens=256,
)

llm = HuggingFacePipeline(pipeline=text_generation_pipeline)

Loading HuggingFaceH4/zephyr-7b-beta...
(First run will download to /content/drive/MyDrive/rag_smart_parking/models, subsequent runs will be fast)


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

...

model-00003-of-00008.safetensors:   0%|          | 0.00/1.98G [00:00<?, ?B/s]

model-00004-of-00008.safetensors:   0%|          | 0.00/1.95G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

Device set to use cuda:0
/tmp/ipython-input-4130118970.py:36: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=text_generation_pipeline)


In [20]:
# --- 7. SETUP RETRIEVAL CHAIN ---
retriever = vector_store.as_retriever(search_kwargs={"k": 3})

prompt_template = """
<|system|>
You are a helpful AI assistant for the Smart Parking App. Answer strictly based on the context provided.
Context: {context}</s>
<|user|>
{question}</s>
<|assistant|>
"""

PROMPT = PromptTemplate(
    template=prompt_template,
    input_variables=["context", "question"]
)

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True,
    chain_type_kwargs={"prompt": PROMPT}
)

In [ ]:
# --- 8. START API SERVER ---
app = Flask(__name__)
CORS(app)

@app.route('/api/chat', methods=['POST'])
def chat_endpoint():
    data = request.json
    query = data.get('query', '')
    if not query: return jsonify({"error": "No query"}), 400

    try:
        result = qa_chain.invoke({"query": query})
        return jsonify({"answer": result['result']})
    except Exception as e:
        return jsonify({"error": str(e)}), 500

!ngrok config add-authtoken 35Y5cyTmsQlev5Kju6boBAyT1kM_DB9soAFCPsH96UWYEE7P
public_url = ngrok.connect(5000).public_url
print(f"\n\n🌟 YOUR PUBLIC URL: {public_url} 🌟\n\n")
app.run(port=5000)

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


🌟 YOUR PUBLIC URL: https://grouseless-nonphysically-craig.ngrok-free.dev 🌟


 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [14/Dec/2025 16:11:08] "POST /api/chat HTTP/1.1" 200 -
